In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from pandas import Series, DataFrame
import math

In [3]:
#知每一年度被納入S&P500的公司

boss=pd.read_excel(r"C:\Users\user\OneDrive\桌面\paper資料\boss_1020.xlsx",sheet_name="boss_1020" )
boss2010=boss[boss["fyear(財報年度)"]==2016]
boss2010
a=boss2010["cik"]
cik_sample_list2=[]
for i in range(0,len(a)):
    b=a.iloc[i]
    if math.isnan(b) == True:
        pass
    else:
        b=int(b)
    cik_sample_list2.append(str(b))    
cik_sample_list=np.unique(cik_sample_list2).tolist()



In [4]:
# 此處以跑前100間為例
endpoint=r"https://www.sec.gov/cgi-bin/browse-edgar"
df2=pd.DataFrame({"cik":["aaa"]})
for j in range(0,100):
#     print(j)
    cik=cik_sample_list[j]
    year="-16"
    param_dict={"action":"getcompany",
           "CIK":cik,
           "type":"10-k",
           "dateb":"20190101",
           "owner":"exclude",
           "start":"",
           "output":"",
           "count":'100'}
    respo=requests.get(url=endpoint,params=param_dict)
#response.url
    urlc=respo.url


    headers = {
        'User-Agent': 'My User Agent 1.0',
        'From': 'youremail@domain.com'  
    }


    response = requests.get(urlc,headers=headers).content
    report_soup=BeautifulSoup(response)
    
    base_securl="https://www.sec.gov"

    link_list=[]
    for link in report_soup.find_all('a'):
        linka=link.get('href')
        link_list.append(linka)    

    dataa = [i for i,x in enumerate(link_list ) if x.find(cik) != -1  & x.find(year) != -1 ] 
    dataanumall=len(dataa)

    linkcik=[]
    for i in range(0,dataanumall):
        dataanum=dataa[i]
        linksam=link_list[dataanum]
        linkcik.append(linksam)


    datab = [i for i,x in enumerate(linkcik ) if x.find("index.htm") != -1 ] 
    databnumall=len(datab)
    linkyear=[]
    
    for i in range(0,databnumall):
        databnum=datab[i]
        linkysam=linkcik[databnum]
        linkyear.append(linkysam)
    
    if linkyear==[]:
        cksb = [cik]
        cksa = [val for val in cksb for i in range(1)]
        df=pd.DataFrame({"cik":cksa})
        df2 = pd.concat([df,df2],axis=0)
    
    else:
        url2=base_securl+linkyear[0]


        response2 = requests.get(url2,headers=headers).content
        report_soup2=BeautifulSoup(response2)
        report_soup2

        link_list2=[]
        for link in report_soup2.find_all('a'):
            linka2=link.get('href')
            link_list2.append(linka2)
    
        data2 = [i for i,x in enumerate(link_list2) if x.find('10-k') != -1  ] 
        data3 = [i for i,x in enumerate(link_list2) if x.find('10k') != -1  ] 

        if data2==[]:
            data4=data3
        elif data3==[]:
            data4=data2
        else:
            cksb = [cik]
            cksa = [val for val in cksb for i in range(1)]
            df=pd.DataFrame({"cik":cksa})

        dataanumall2=len(data4)

        linkcik2=[]
        for i in range(0,dataanumall2):
            dataanum2=data4[i]
            linksam2=link_list2[dataanum2]
            linkcik2.append(linksam2)


        datab2 = [i for i,x in enumerate(linkcik2 ) if x.find("htm") != -1 ] 

        databnumall2=len(datab2)
        linkff=[]
    
        for i in range(0,databnumall2):
            databnum2=datab2[i]
            linkysam2=linkcik2[databnum2]
            linkff.append(linkysam2)
        if linkff==[]:
            cksb = [cik]
            cksa = [val for val in cksb for i in range(1)]
            df=pd.DataFrame({"cik":cksa})
            df2 = pd.concat([df,df2],axis=0,ignore_index = True)
        else:
            

            url3=base_securl+linkff[0]
            response3 = requests.get(url3,headers=headers).content
            report_soup3=BeautifulSoup(response3,"html")
            ppp=report_soup3.div
            if ppp is None:
                ft="NA"
            else:
                epp=ppp.find_all_next(string=True)
                bbb=[]
                

                emps3 = [i for i,x in enumerate(epp) if x.find('employees') != -1 or x.find('Employees') != -1] 
                emst3=len(emps3)
                for i in range(0,emst3):
                    ii = emps3[i]
                    iii = epp[ii]
                    if len(iii) > 20:
                        bbb.append(iii)
                    else:
                        ccc = epp[ii-10:ii+10]
                        cc = " ".join(ccc)
                        bbb.append(cc)
                df = pd.DataFrame(bbb)
                ddf = df.T
                ddf.insert(0,"cik",cik)
                ddf.insert(1,"10-k",url3)
                df2 = pd.concat([ddf,df2],axis=0,ignore_index = True)
df2


,cik,10-k,0,1,2,3,4,5,6,7,...,68,69,70,71,72,73,74,75,76,77
0,1094093,https://www.sec.gov/Archives/edgar/data/17797/...,Commercial Portfolio relies on wind and solar ...,", Duke Energy had a total of 29,188 employees ...","Certain events, such as an aging workforce, mi...","In 2015, Duke Energy advanced a number of impo...",Operational excellence through engagement with...,Management evaluates financial performance in ...,Duke Energy elects to amortize net actuarial g...,Duke Energy maintains non-contributory defined...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,109380,https://www.sec.gov/Archives/edgar/data/109380...,. The Parent and its subsidiaries (collectivel...,The Board of Directors of the Parent has imple...,The regulation of incentive compensation under...,The bank regulatory agencies have published gu...,We may encounter significant adverse developme...,limits the manner and amount in which compensa...,Represents common shares acquired from employe...,Full-time equivalent employees,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1093557,https://www.sec.gov/Archives/edgar/data/109355...,We plan to develop future generations of techn...,"We also rely on trade secrets, technical know-...",believes we or any of our contract manufactur...,21 International Regulation International sale...,full-time employees and,contract and temporary employees. Approximately,full-time employees are engaged in research a...,"in selling, general and administrative functi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1091667,https://www.sec.gov/Archives/edgar/data/109166...,Charter offers basic coax service primarily to...,Charter offers fiber or complex services to me...,simplicity for both our customers in understan...,"As the Internet has matured, it has become the...",", we had approximately 23,800 full-time equiva...",", approximately 60 of our employees were repre...","If we are unable to retain key employees, our ...","Our operational results have depended, and our...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1090872,https://www.sec.gov/Archives/edgar/data/109087...,Our order fulfillment and supply chain organiz...,". In addition, many of our employees, contract...","Additionally, we must comply with complex fore...",Our future success depends partly on the conti...,"In the normal course of business, we frequentl...",A successful divestiture depends on various fa...,We rely on several centralized information tec...,We account for share-based awards in accor...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,1001250,https://www.sec.gov/Archives/edgar/data/100125...,We have been controlled by the Lauder family s...,"During fiscal 2016, we also completed the depl...",We believe that we are an industry leader in t...,Research and development costs are expensed as...,"C, Bobbi Brown, La Mer, Aveda, Jo Malone Londo...","At June 30, 2016, we had approximately 46,000 ...",The Board of Directors has developed corporate...,difficulties in integrating acquired operation...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,1001082,https://www.sec.gov/Archives/edgar/data/100108...,"quality installations, upgrades, and in-home r...","We had approximately 18,000 employees at Decem...",We have adopted a written code of ethics that ...,Technological innovation is important to our s...,for our Sling TV services. If EchoStar is un...,"In addition, we may not be able to successfull...",Our employees participate in the DISH Network ...,million shares of Class A common stock which ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,1001039,https://www.sec.gov/Archives/edgar/data/100103...,We maintain information necessary to conduct o...,"employees, our profitability is substantially...",A significant number of employees in various o...,Disney Key Employees Retirement Savings Plan,"Amendments dated April 30, 2015 to the Amended...",The Company maintains pension and

In [5]:
co = df2.shape[0]
ro = df2.shape[1]
html10k = df2.iloc[:,[0,1]]
ciklist = df2.iloc[:,0]

In [6]:
# 資料清洗(將標點符號刪除 轉小寫)

def isNaN(num):
    return num != num
bbb=[]

line = "Huang's work is part-time?,no,he is a student."
pun = """！？｡＂＃＄％＆＇（）＊＋－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞–—‘'‛“”„‟…‧﹏"""
re_pun = "[{}] ".format(pun)


newline = re.sub("[,\!?;'/-:’-]", "", line)
newline
#re_pun


for j in range(0,co):
    for i in range(2,ro):
        b = df2.iloc[j,i]
        if isNaN(b) == False:
            #out = re.sub("(“[,\!?;'/-:’-]”)","",b)
            out = re.sub("[(“”),\!?;'/-:’-]","",b)
            
            output = out.lower()
            bbb.append(output)


        else:
            pass
bbb
bdf = pd.DataFrame(bbb)
inti = pd.DataFrame()
for i in range(0,co):
    cik = df2.iloc[i,0]
    ccc = df2.iloc[i,2:]
    ccc = ccc.dropna()
    cc = len(ccc)
    cik = [cik]*cc
    cikt = pd.DataFrame(cik)
    inti = pd.concat([inti,cikt],axis=0,ignore_index = True)

data = pd.concat([inti,bdf],axis=1,ignore_index = True)
data = data.dropna()
data

,0,1
0,1094093,commercial portfolio relies on wind and solar ...
1,1094093,duke energy had a total of employees on its ...
2,1094093,certain events such as an aging workforce mism...
3,1094093,in duke energy advanced a number of important...
4,1094093,operational excellence through engagement with...
...,...,...
2048,1000228,stockbased awards are provided to certain empl...
2049,1000228,we offer qualified k plans to substantially al...
2050,1000228,we offer an unfunded nonqualified supplemental...
2051,1000228,during we began to offer a deferred compensat...


In [7]:
datacombind = pd.DataFrame(ciklist)
emda = []
for i in range(0,len(ciklist)):
#     print(i)
    a = ciklist[i]

    das = data[data.iloc[:,0]==a]
    dbs = das.iloc[:,1].values.tolist()
    dcs = " ".join(dbs)
    emda.append(dcs)
datacombind.insert(1,"data",emda)
datacombind 




,cik,data
0,1094093,commercial portfolio relies on wind and solar ...
1,109380,. the parent and its subsidiaries collectively...
2,1093557,we plan to develop future generations of techn...
3,1091667,charter offers basic coax service primarily to...
4,1090872,our order fulfillment and supply chain organiz...
...,...,...
86,1001250,we have been controlled by the lauder family s...
87,1001082,quality installations upgrades and inhome repa...
88,1001039,we maintain information necessary to conduct o...
89,1000228,. our operating entities and employees engage...


In [13]:
ddd = pd.merge(datacombind,html10k)
ddd
ddd.to_excel("2016_1.xlsx")